Build and Push the docker image to Google Container Registry

In [14]:
# Step-1 : Build Docker Image 
! docker build -t cc-fraud-detection .

# Step-2 : Tag Docker Image 
! docker tag cc-fraud-detection gcr.io/{your-project-id}/cc-fraud-detection

# Step-3 : Push docker image to Artifact Registry 
! docker push gcr.io/{your-project-id}/cc-fraud-detection

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (1/2)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context:                                               0.0s
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.0s
[+] Building 0.4s (2/3)              

In [11]:
from google.cloud import aiplatform
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path-to-sa-credentials/vertexai-sa.json"
aiplatform.init(project='your-project-id', location='us-central1')

Create a New Service Account in GCP before executing the below gcloud commands    

In [ ]:
# Execute the below from your local system after "gcloud auth login "

!!gcloud services enable cloudresourcemanager.googleapis.com

!gcloud projects add-iam-policy-binding {your-project-id} \
    --member=serviceAccount:vertex-ai-sa@{your-project-id}.iam.gserviceaccount.com \
    --role=roles/aiplatform.customCodeServiceAgent

!gcloud projects add-iam-policy-binding {your-project-id} \
    --member=serviceAccount:vertex-ai-sa@{your-project-id}.iam.gserviceaccount.com \
    --role=roles/aiplatform.admin

!gcloud projects add-iam-policy-binding {your-project-id} \
    --member=serviceAccount:vertex-ai-sa@{your-project-id}.iam.gserviceaccount.com \
    --role=roles/storage.objectAdmin

Train the model in Vertex AI using the container image uri from GCP container registry

In [17]:
display_name = 'fraud-detection-traning'
container_uri = 'gcr.io/{your-project-id}/fraud-model'
bucket = 'vertex-ai-udemy'

container_deployment = aiplatform.CustomContainerTrainingJob(display_name=display_name,
                                               container_uri=container_uri,
                                               staging_bucket=bucket)

model = container_deployment.run(replica_count=1,machine_type = "n1-standard-4",
                                accelerator_type= "ACCELERATOR_TYPE_UNSPECIFIED",
                                accelerator_count=1)



Training Output directory:
gs://vertex-ai-udemy/aiplatform-custom-training-2022-12-24-20:57:25.772 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5813977931583586304?project=470181514403
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6011080784025223168?project=470181514403
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/5813977931583586304 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/5813977931583586304 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/5813977931583586304 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/470181514403/locations/us-central1/trainingPipelines/5813977931583586304 current state:
PipelineState.PIPEL

Upload Model from Artifact Registry to Vertex AI Model Registry 

In [5]:
artifact_uri = "gs://your-bucket-name/model_output/model"

serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest' # Dont change this 
fraud_detection_model = aiplatform.Model.upload(display_name='fraud-model',
                                  artifact_uri=artifact_uri,
                                  serving_container_image_uri=serving_container_image_uri)

Creating Model
Create Model backing LRO: projects/470181514403/locations/us-central1/models/8647511617900118016/operations/1672118935421452288
Model created. Resource name: projects/470181514403/locations/us-central1/models/8647511617900118016@1
To use this Model in another session:
model = aiplatform.Model('projects/470181514403/locations/us-central1/models/8647511617900118016@1')


In [6]:
endpoint = fraud_detection_model.deploy(machine_type="n1-standard-4")

Creating Endpoint
Create Endpoint backing LRO: projects/470181514403/locations/us-central1/endpoints/9166817556768489472/operations/1336600763182350336
Endpoint created. Resource name: projects/470181514403/locations/us-central1/endpoints/9166817556768489472
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/470181514403/locations/us-central1/endpoints/9166817556768489472')
Deploying model to Endpoint : projects/470181514403/locations/us-central1/endpoints/9166817556768489472
Deploy Endpoint model backing LRO: projects/470181514403/locations/us-central1/endpoints/9166817556768489472/operations/850212003426336768


FailedPrecondition: 400 Model server terminated: model server container terminated: exit_code: 	 139
reason: "Error"
started_at {
  seconds: 1671818572
}
finished_at {
  seconds: 1671818578
}
. Model server logs can be found at https://console.cloud.google.com/logs/viewer?project=470181514403&resource=aiplatform.googleapis.com%252FEndpoint&advancedFilter=resource.type%3D%22aiplatform.googleapis.com%2FEndpoint%22%0Aresource.labels.endpoint_id%3D%229166817556768489472%22%0Aresource.labels.location%3D%22us-central1%22.